In [17]:
%matplotlib inline
#%reset
import parcels
from parcels import ParticleFile, GeographicPolar
import numpy as np
import pandas as pd
import xarray as xr
#import matplotlib
import datetime as dati
from operator import attrgetter
from math import sqrt, pi, cos
import warnings as wr

## Coords are always the same

In [18]:
path = "D:\\OceanParcels\\"
csname = path + "data\\coordinates.nc"
cs = xr.open_dataset(csname).load()
lon_grid = cs.lon_grid.values
lat_grid = cs.lat_grid.values
dep_grid = cs.dep_grid.values
cs.close()

npart = 8 # will be squared, then doubled

# Releases 30 particles at grid of lats, lons and deps every ten hours
lon_range = np.linspace(lon_grid[4,4], lon_grid[37,38], num = npart)
lat_range = np.linspace(lat_grid[4,4], lat_grid[37,38], num = npart)

# Create a grid of particles
plons = np.repeat(lon_range, npart)
plons = np.repeat(plons, 2)
plats = np.tile(lat_range, npart)
plats = np.repeat(plats, 2)

def DeleteParticle(particle, fieldset, time): 
            particle.delete() # Deletes particle if it's out of bounds

del cs, lon_range, lat_range

# All in one go!

In [19]:
scen = np.array(["S0", "S1", "S2", "S3", "S4", "S5", "S6"])
mon = np.array(["may", "jun", "jul", "aug", "sep", "oct"])

ds = xr.open_dataset(path + "data\\raw\\reindexed_sep.nc").load()
#times_secs = ds.times_secs.values
#U_atten = ds.U_atten.values
#V_atten = ds.V_atten.values
#W = ds.W.values
ds.close()
ds

<xarray.Dataset>
Dimensions:     (t: 4320, k: 22, j: 42, i: 43)
Coordinates:
    times       (t) float64 4.626e+03 4.626e+03 ... 4.656e+03 4.656e+03
    times_secs  (t) float64 7.949e+06 7.949e+06 7.95e+06 ... 1.063e+07 1.063e+07
Dimensions without coordinates: t, k, j, i
Data variables:
    U           (t, k, j, i) float32 -0.1817 -0.1809 -0.18 ... 0.0 0.0 0.0
    V           (t, k, j, i) float32 -0.1219 -0.1214 -0.1221 ... 0.0 0.0 0.0
    W           (t, k, j, i) float32 -8.224e-06 6.158e-05 -6.19e-06 ... 0.0 0.0
    Uav         (t, j, i) float32 0.005436 0.005578 0.005715 ... 0.04389 0.04431
    Vav         (t, j, i) float32 0.005965 0.005911 0.005887 ... 0.02767 0.02495

Uav and Vav are the depth-averaged velocity components, but u_c and u_b apply to below and canopy speeds respectively... how to apply them proportionately? Is it simply proportional to h_c/z?

In [20]:
sc = 0
mo = 0
#for mo in range(6):

dfile = path + "data\\final grids\\final_" + mon[mo] + "_" + scen[sc] + "_1.nc"
zarrfile = path + "outputs\\2D_" + scen[mo] + "_" + mon[sc] + ".zarr"

ds = xr.open_dataset(dfile).load()
times_secs = ds.times_secs.values
Uav = ds.Uav.values
Vav = ds.Vav.values
ds.close()

# Define fieldset
vals = {'U': Uav, 'V': Vav}

dims = {'U': {'lon': lon_grid, 'lat': lat_grid, 'time': times_secs},
        'V': {'lon': lon_grid, 'lat': lat_grid, 'time': times_secs}}

fset = parcels.FieldSet.from_data(data = vals, dimensions = dims,
                                  chunksize = [512], # allow_time_extrapolation = True,
                                  gridindexingtype = 'nemo')

del Uav, Vav, ds
print("Fieldset has grid type: %s" %fset.U.grid.gtype)

Fieldset has grid type: GridCode.CurvilinearZGrid


In [21]:
pset = parcels.ParticleSet.from_list(fieldset = fset, pclass = parcels.ScipyParticle, 
                                     lon = plons, lat = plats,
                                     time = np.ones(npart*npart*2)*fset.gridset.dimrange('time')[0],
                                     repeatdt = dati.timedelta(hours = 14).total_seconds())
pkern = pset.Kernel(parcels.AdvectionRK4)
pset.populate_indices()
outfile = pset.ParticleFile(name = zarrfile, outputdt = dati.timedelta(hours = 1))

pset.execute(pkern, output_file = outfile, dt = dati.timedelta(minutes = 30), 
             recovery = {parcels.ErrorCode.ErrorOutOfBounds: DeleteParticle})

del pset, fset, pkern

Correct cell not found after 100000 iterations
Correct cell not found after 100000 iterations
Correct cell not found after 100000 iterations
Correct cell not found after 100000 iterations
INFO: Output files are stored in D:\OceanParcels\outputs\2D_S0_may.zarr.


  0%|          | 0/2677800.0 [00:00<?, ?it/s]

  1%|          | 28800.0/2677800.0 [00:00<00:48, 55137.78it/s]

  1%|▏         | 36000.0/2677800.0 [00:01<02:01, 21715.78it/s]

  1%|▏         | 39600.0/2677800.0 [00:01<02:28, 17782.68it/s]

  2%|▏         | 43200.0/2677800.0 [00:02<02:59, 14686.74it/s]

  2%|▏         | 46800.0/2677800.0 [00:02<03:22, 12983.61it/s]

  2%|▏         | 50400.0/2677800.0 [00:03<04:17, 10214.57it/s]

  2%|▏         | 54000.0/2677800.0 [00:04<06:03, 7224.16it/s] 

  2%|▏         | 57600.0/2677800.0 [00:04<06:49, 6391.30it/s]Correct cell not found after 100000 iterations
Correct cell not found after 100000 iterations
Correct cell not found after 100000 iterations


  2%|▏         | 57600.0/2677800.0 [00

KeyboardInterrupt: 

In [ ]:
%%script echo Skipped!

import sys, glob

# These are the usual ipython objects, including this one you are creating
ipython_vars = ['In', 'Out', 'exit', 'quit', 'get_ipython', 'ipython_vars']

# Get a sorted list of the objects and their sizes
sorted([(x, sys.getsizeof(globals().get(x))) for x in dir() if not x.startswith('_') and x not in sys.modules and x not in ipython_vars], key=lambda x: x[1], reverse=True)